<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/veronica-thesis/blob/master/malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.utils import Sequence
#from tensorflow.python.keras.utils import data_utils
import math
import os
import pandas
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv3D,Conv2D,InputLayer,MaxPooling3D,Dropout,Flatten,MaxPooling2D,BatchNormalization,LayerNormalization


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:

def conv(x):
    if x=='??':
        return -1
    else:
        return int(x,16)


def getImage(filename):
    f=open(filename)
    rest=f.read()
    lines=rest.splitlines()
    img=[]
    for line in lines:
        el=line.split()
        el=list(map(conv,el))
        if(len(el)==17):
            img.append(el[1:])
    img=np.asarray(img,dtype=np.uint8)
    img=Image.fromarray(img,'L')
    img=img.resize((128,128),resample=Image.BOX)
    img=np.array(img)
    img=img.reshape(img.shape[0],img.shape[1],1,1)
    f.close()
    return img

class ImageSequence(Sequence):
    def __init__(self, x_set, y_set,dir, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.dir=dir

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([getImage(dir+filename+".bytes") for filename in batch_x]),batch_y


In [3]:
batch_size=32
data=pandas.read_csv("data/data.1.csv",header=None)
names=data.iloc[:,0].to_numpy()
classes=data.iloc[:,1].to_numpy()-1
##data=pandas.read_csv("data/data.2.csv",header=None)
##names=np.append(names,data.iloc[:,0].to_numpy())
##classes=np.append(classes,data.iloc[:,1].to_numpy)
#dir="c:\\Users\\hikma\\Downloads\\"
dir="data\\train\\"

train_dataset=ImageSequence(names,classes,dir,batch_size)

##data=pandas.read_csv("data/data.6.csv",header=None)
# test dataset
##names=data.iloc[0:500,0].to_numpy()
##classes=data.iloc[0:500,1].to_numpy()-1
##test_dataset=ImageSequence(names,classes,dir,batch_size)
(f,l)=train_dataset.__getitem__(0)
f.shape
l

array([2, 8, 7, 6, 7, 7, 1, 5, 2, 0, 0, 2, 1, 2, 7, 2, 2, 2, 2, 0, 0, 6,
       2, 3, 0, 2, 7, 2, 5, 8, 1, 2], dtype=int64)

In [4]:
f.shape

(32, 128, 128, 1, 1)

In [30]:
def createModelGilbert():
    model = Sequential()
    model.add(InputLayer(input_shape=(128,128,1,1)))
    model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))
   
    model.add(Conv3D(50, kernel_size = (5,5,1), activation = 'relu'))
    model.add(MaxPooling3D(pool_size = (2,2,1)))
    model.add(LayerNormalization())
    
    #model.add(Conv3D(70, kernel_size = (3,3,50), activation = 'relu'))
    #model.add(MaxPooling3D(pool_size = (2,2,1)))
    #model.add(LayerNormalization()) 

    #model.add(Conv3D(70,kernel_size=(3,3,70),activation='relu'))
    #model.add(MaxPooling3D(pool_size = (2,2,1)))
    #model.add(LayerNormalization())


    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(9, activation = 'softmax', name = 'Output'))
    return model


In [31]:
#model=createModel()
#tf.keras.utils.plot_model(model,show_shapes=True)
model=createModelGilbert()
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_12 (Rescaling)     (None, 128, 128, 1, 1)    0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 124, 124, 1, 50)   1300      
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 62, 62, 1, 50)     0         
_________________________________________________________________
layer_normalization_7 (Layer (None, 62, 62, 1, 50)     100       
_________________________________________________________________
flatten_3 (Flatten)          (None, 192200)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               49203456  
_________________________________________________________________
Output (Dense)               (None, 9)               

## Optimization

Keras can use many optimization method. In this notebook we use the __Adam__ method which can be described loosely as __adaptive__ gradient descent.

Also since the labels are __NOT__ in one_hot_encoding we use the "Sparse" version of the crossentropy loss: __SparseCategoricalCrossentropy__. Finally, if we don't specify from_logits=False then the loss function would compute softwmax before computing the loss. Since we are computing softwmax in our model already we turn this step off by specifying from_logits=False

In [27]:
# if we don't use softmax in the last layer, i.e. if the output of the
# model is NOT probabilities then use from_logits=True
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

#history = model.fit(img_train,label_train, batch_size=128,epochs=10, 
#                    validation_data=(img_test, label_test))
history=model.fit(train_dataset,batch_size=batch_size,epochs=3)


Epoch 1/3
 7/63 [==>...........................] - ETA: 28:42 - loss: 14.8080 - accuracy: 0.1260

### Testing the Accuracy

In [10]:
data=pandas.read_csv("data/data.6.csv",header=None)
# test dataset
n=200
names=data.iloc[0:n,0].to_numpy()
classes=data.iloc[0:n,1].to_numpy()-1
test_dataset=ImageSequence(names,classes,dir,n)
_,test_accuracy=model.evaluate(test_dataset)

1/1 [==============================] - 87s 87s/step - loss: 0.4443 - accuracy: 0.8850
